# Decision Tree Assignment1

김효은

---

## Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
pd_data.head()

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


## 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [4]:
def get_gini(df, label):
    
    # df에서 label의 비율 구하기
    probability = df[label].value_counts() / df.shape[0]
    probability = probability.values

    # gini index 구하기 (gini = 1-sum(pi^2))
    gini = 1
    for p in probability:
        gini = gini - (p)**2
        
    return gini

In [5]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

## 2. Feature의 Class를 이진 분류로 만들기
ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [6]:
import itertools # 변수의 모든 클래시 조합을 얻기 위해 itertools 불러오기
def get_binary_split(df, attribute):
    
    unique = df[attribute].unique() # df[attribute]에서 unique한 값 추출
    nunique = len(unique) # unique한 값의 개수

    result = [] # binary split한 결과를 담을 리스트
    for i in range(1, nunique): # 1부터 nunique-1개까지
        combination = map(list, itertools.combinations(unique, i)) # unique에서 i개를 추출할 수 있는 모든 경우의 수
        for comb in combination:
            result.append(comb)
    
    return result 

In [7]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

## 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [8]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {} # gini index를 담을 dict
    binary_split = get_binary_split(df, attribute) # attribute에 대해서 binary split
   
    for binary in binary_split:
        # binary가 들어가 있는 데이터프레임을 구하기 위한 index
        index1 = np.array([False] * len(df[attribute]))
        for value in binary:
            index1 = index1 | (df[attribute] == value).values
            
        # binary가 들어가 있지 않는 데이터프레임을 구하기 위한 index
        index2 = np.array([True] * len(df[attribute]))
        for value in binary:
            index2 = index2 & (df[attribute] != value).values
        
        # 데이터를 index에 따라 분리한 후 각각의 gini index를 구한다.
        gini1 = get_gini(df.loc[index1, [attribute, label]], label)
        gini2 = get_gini(df.loc[index2, [attribute, label]], label)
        
        # binary split 했을 때 각 split에 속하는 데이터의 개수
        d1 = sum(index1); d2 = sum(index2)
        d  = d1+d2

        # 최종 gini index
        gini = (d1/d)*gini1 + (d2/d)*gini2
        
        # result dict에 key는 binary, value는 gini를 저장
        result[ tuple(binary) ] = gini
        
    return result

In [9]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?  
결과를 확인해보도록 하겠습니다.

In [10]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


## 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
- 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.
- 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [11]:
# 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.
# 문제1 답안
get_binary_split(pd_data, 'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [12]:
# 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
# 문제2 답안

def get_min_gini(df, feature_names, target_name):
    min_gini = ['feature_name', 'bianry', 1]

    for feature in feature_names:
        gini = get_attribute_gini_index(df, feature, target_name)
        if min_gini[2] > min(gini.values()):
            min_gini[0] = feature
            min_gini[1] = min(gini)
            min_gini[2] = min(gini.values())
    return min_gini

In [16]:
feature_names = pd_data.columns[:-1]
min_gini = get_min_gini(pd_data, feature_names, 'class_buys_computer')
print(min_gini)

# age feature를 middle_aged와 middle_aged로 아닌 경우로 분류했을 때 gini index가 0.357로 가장 낮았다.

['age', ('middle_aged',), 0.35714285714285715]


In [14]:
# 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 
# 해당 변수의 Gini index를 제시해주세요.
#문제3 답안

In [17]:
# 위의 결과를 받아옴
attribute = min_gini[0]
binary    = min_gini[1]

# age가 middle_aged인 데이터의 인덱스
index1 = np.array([False] * len(pd_data[attribute]))
for value in binary:
    index1 = index1 | (pd_data[attribute] == value).values

# age가 middle_aged가 아닌 데이터의 인덱스
index2 = np.array([True] * len(pd_data[attribute]))
for value in binary:
    index2 = index2 & (pd_data[attribute] != value).values

# age가 middle_aged인 데이터와 middle_aged가 아닌 데이터로 분리
df_middle_aged = pd_data.loc[index1, ]
df_non_middle_aged = pd_data.loc[index2, ]

**age가 middle_aged인 데이터인 경우**

In [18]:
# age가 middle_aged인 데이터인 경우
# 여기서 주의할 점은 df_middle_aged의 데이터셋의 경우 age가 middle_aged인 데이터만 있으므로,
# gini index를 구할 때 age feature는 제외하고 구해야 한다는 점

# 그런데 df_middle_aged를 보면 이미 class_buys_computer가 'yes'인 데이터만 모여있다.
# 따라서 더이상 branch split을 하지 않아도 된다.
# 또한 모든 경우에 대해 gini index를 구해보아도 모두 0으로 동일하다는 것을 확인할 수 있다.
# 즉, 이 데이터셋은 이미 불순도가 0% 라는 것!

features = df_middle_aged.columns[1:-1]
for feature in features:
    print( get_attribute_gini_index(df_middle_aged, feature, 'class_buys_computer') )

{('high',): 0.0, ('low',): 0.0, ('medium',): 0.0, ('high', 'low'): 0.0, ('high', 'medium'): 0.0, ('low', 'medium'): 0.0}
{('no',): 0.0, ('yes',): 0.0}
{('fair',): 0.0, ('excellent',): 0.0}


In [19]:
df_middle_aged

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


**age가 middle_aged가 아닌 데이터인 경우**

In [20]:
# age가 middle_aged가 아닌 데이터인 경우
# 이 데이터는 age가 youth, senior인 데이터셋이기 때문에 gini index구할 때 age feature를 고려할 수 있다.
# (CART 알고리즘은 이진분류를 적용하기 때문에 그 feature의 class가 3개 이상인 경우 이전에 사용된 feature가 다시 고려될 수 있다!)

# age가 middle_aged가 아닌 데이터인 경우 student 여부에 따라 분류했을 때 gini inex가 0.32 정도로 가장 낮았다.
# 따라서 다음 branch split은 student 여부가 되겠다.
get_min_gini(df_non_middle_aged, feature_names, 'class_buys_computer')

['student', ('no',), 0.31999999999999984]